<a href="https://colab.research.google.com/github/YuliiaPantana/PRA3024/blob/master/MyFile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import sys
!{sys.executable} -m pip install pycbc ligo-common --no-cache-dir 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 KB 114.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 134.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 296.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.2/201.2 KB 252.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 217.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 KB 156.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 MB 160.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from pycbc.frame import read_frame
from pycbc.filter import highpass_fir, lowpass_fir
from pycbc.waveform import get_fd_waveform
from pycbc.psd import welch, interpolate
from pycbc.catalog import Merger
import pylab
from pycbc.types import  FrequencySeries, TimeSeries

#from scipy import interpolate

import numpy as np
from pycbc import frame
import pycbc
from pycbc.filter import resample_to_delta_t, highpass
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from scipy.stats import norm 
from pycbc.detector import Detector
from scipy.interpolate import interp1d

import numpy as np



In [5]:
d = np.load('/content/drive/MyDrive/GW_noisedata-2.npy') #name of the data file noise_ts_4096Hz.npy
dt = d[:, 0]
data_H1 = pycbc.types.TimeSeries(d[:, 1], delta_t = dt[1]- dt[0]) #this is to comvert data into GW timeSeries
data_L1 = pycbc.types.TimeSeries(d[:, 2], delta_t = dt[1]- dt[0]) #this is to comvert data into GW timeSeries
merger = Merger("GW170817")
strain, stilde = {}, {}
strain['H1'] = resample_to_delta_t(highpass(data_H1, 15.0), 1.0/2048)
strain['L1'] = resample_to_delta_t(highpass(data_L1, 15.0), 1.0/2048)

strain['H1'] = strain['H1'].crop(4, 4)
strain['L1'] = strain['L1'].crop(4, 4)

stilde['H1'] = strain['H1'].to_frequencyseries()
stilde['L1'] = strain['L1'].to_frequencyseries()


In [15]:
#Problem 1, Part 2 (Matched Filtering. Finding template with maximum SNR. Plotting SNR versus total mass.)

from pycbc.waveform import get_fd_waveform
from pycbc.psd import aLIGOZeroDetHighPower
from pycbc.filter import matched_filter
from pycbc.conversions import mass1_from_mchirp_q
from pycbc.types import TimeSeries
from pycbc.noise import noise_from_psd
import numpy

#Here the masses a defined to be in range from 1 to 3 solar masses with step 0,1 
mass1 = numpy.arange(1 ,3 ,0.10 ) 
mass2 = mass1

# Variables to store when we've found the max
hmax, smax, tmax, mmax, nsnr = None, {}, {}, 0, 0
snrs = {}
ifo = 'H1'
# Defining the duration and sampling rate of the data
duration = 32.0
sampling_rate = 4096.0
psds = {}
for ifo in ['H1']:
    flow = 10.0
    delta_f = 1.0 / duration
    psd = pycbc.psd.aLIGOZeroDetHighPower(delta_f=delta_f,length=int(duration*4096),low_freq_cutoff=flow)
    psds[ifo] = psd
    
#Here for detector H1 matched filtering will be performed and the tamplate for which SNR is maximum will be found and assigned to dictionary 

Snr_list = []
for i in range(len(mass1)):
    #Here the waveform will be generated with a given masses that change according to cycle and mass range defined above; assumed equal mass, nonspinning
 
    m = mass1[i] 
    hp, hc = get_fd_waveform(approximant="TaylorF2",
                     mass1=m,
                     mass2=m,
                     delta_f= 1.0/stilde[ifo].delta_f, 
                     f_lower=15.0)

    # Here the lenth of the tamplate will be matched with length of data 'strain'
    hp.resize(len(stilde[ifo])) 
    #Here hte waveform is shifted by a certain amount of time and the end of the waveform is wrapped around to the beginning
    template = hp.cyclic_time_shift(hp.start_time)
    #Here the matched filter is performed by using the waveform as a template for the signal that should be filtered from noise
    snr = matched_filter(template, stilde[ifo], psd=psds[ifo], low_frequency_cutoff=15.0) #calling matched filter, template is K(t)

    #Here SNR time series are croped by removing the first 4 and last 4 seconds of data
    #In this way, discarding any signal at the beginning or end of the data that could be corrupted by filtering  
    snr_H1 = snr.crop(4 + 4, 4)
    #Here the maximum absolute value of the SNR time series will be calculated and appended to the Snr_list and assigned to disctionary
    Snr_list.append(max(abs(snr_H1)))
snrs[ifo] = Snr_list

#Here the plot which shows SNR versus total mass of the system
pylab.figure(figsize=[10, 4])
pylab.plot(mass1*2, snrs['H1'], label= 'H1')
pylab.legend('True')
pylab.ylabel('Signal-to-noise')
pylab.xlabel('mass')
pylab.show()


ValueError: ignored

In [14]:
#Problem 1, Part 2 (The graph of the signal vesus time for the best tamplate)
#The best template was defined as the waveform with the mass for which the SNR was maximum
#In our case the mass for which SNR was maximum was '1'

m = 1

#Here the waveform for mass 1 is generated
conditioned = strain['H1']
hp, hc = get_td_waveform(approximant="TaylorF2",
                     mass1=m,
                     mass2=m,
                     delta_t=conditioned.delta_t,
                     f_lower=15.0)

# Here the data is resised to match the length of the template
hp.resize(len(conditioned))

#Here the data for the waveform will be shifted, the waveform vector will be rotated by foxed amount 
template = hp .cyclic_time_shift(hp.start_time)
#Here the strain filtered based on the best fitted tamplate versus the time will be plotted
pylab.plot(hp.sample_times, template)
pylab.xlabel('time')
pylab.ylabel('stilde')
pylab.show()

NameError: ignored

In [ ]:
#Problem 1, Part 2 (Q-transform)
#Here the Q-transformation() will be performed to transform a signal from the time domain to the time-frequency domain
from pycbc.filter import sigma
# At first align the proposed signal should be aligned with the data

# Shift the template to the peak time
dt = time - conditioned.start_time
#Here a cyclic time shift of the template waveform by dt is performed
#In this way,the peak of the template is aligned with the peak of the data
aligned = template.cyclic_time_shift(dt)

#Here the template is scaled so that it would have SNR 1 in this data
aligned /= sigma(aligned, psd=psds[ifo], low_frequency_cutoff=20.0)

#Here This the amplitude and phase of the template waveform are scaled to the peak SNR value
#And time-domain template is conveted to the frequency domain 
aligned = (aligned.to_frequencyseries() * snrp).to_timeseries()
#Here the start time of the aligned template is set to be the same as the start time of the conditioned data
aligned.start_time = conditioned.start_time

#Here the the aligned template waveform is substracted from the conditioned data.
#In this way, the part of the data that can be explained by the template waveform is removed, so that the remaining signal can be seen 

subtracted = conditioned - aligned

#This a Q-transform isperformed for the substracted signal

for data, title in [(conditioned, 'Original H1 Data'),
                    (subtracted, 'Signal Subtracted from H1 Data')]:

    t, f, p = data.whiten(4, 4).qtransform(.001,
                                                  logfsteps=100,
                                                  qrange=(8, 8),
                                                  frange=(20, 512))
#Here frequency versus time will be plotted

    pylab.figure(figsize=[15, 3])
    pylab.title(title)
    pylab.pcolormesh(t, f, p**0.5, vmin=1, vmax=6)
    pylab.yscale('log')
    pylab.xlabel('Time (s)')
    pylab.ylabel('Frequency (Hz)')
    pylab.xlim(merger.time - 2, merger.time + 1)
    pylab.show()


In [ ]:
#Problem 1, Part 3 (Match of two waveforms)

from​ ​pycbc.waveform​ ​import​ get_td_waveform 
from​ ​pycbc.filter​ ​import​ match 
from​ ​pycbc.psd​ ​import​ aLIGOZeroDetHighPower 
 
f_low ​=​ ​30 
sample_rate ​=​ ​4096 
 
# Here two waveforms will be generated in order to calculate the match between them 

#Here the waveform generated from a binary with masses m1=m2=10 solar masses will be generated
hp, hc ​=​ get_td_waveform(approximant​=​"EOBNRv2"​, 
                        
                         mass1​=​10​, 
                         mass2​=​10​, 
                         f_lower​=​f_low, 
                         delta_t​=​1.0​/​sample_rate)
#Here the waveform generated assuming m1<= 5 solar masses, m2<= 15 solar masses

mass1 = numpy.arange(0 ,5 ,0.033 ) 
mass2 = numpy.arange(0 ,15 ,0.10 ) 

 
for i in range(len(mass1)):
    #Here the waveform will be generated with a given masses that change according to cycle and mass range defined above; assumed equal mass, nonspinning
 
    m1 = mass1[i]
    m2 = mass2[i] 
    sp, sc = get_td_waveform(approximant="TaylorT4",
                     mass1=m1,
                     mass2=m2,
                     f_lower​=​f_low, 
                     delta_t​=​1.0​/​sample_rate)


 
# Here the waveforms are resized to the same length 
tlen ​=​ ​max​(​len​(sp), ​len​(hp)) 
sp​.​resize(tlen) 
hp​.​resize(tlen) 
 
# Generate the aLIGO ZDHP PSD 
delta_f ​=​ ​1.0​ ​/​ sp​.​duration 
flen ​=​ tlen​//​2​ ​+​ ​1 
psd ​=​ aLIGOZeroDetHighPower(flen, delta_f, f_low) 
 

#Here the match btween two waveforms is calculated
m, i ​=​ match(hp, sp, psd​=​psd, low_frequency_cutoff​=​f_low)

In [28]:
#Problem 2, Part 1
#Loading the data
import pycbc.types
import numpy
from pycbc.waveform import get_td_waveform 

d = np.load('/content/drive/MyDrive/noise_ts_4096Hz.npy')
time = d[:, 0]
strain = d[:,1]
dt = time[1]-time[0]
data=pycbc.types.TimeSeries(d[:,1], delta_t=dt)
ifo = 'data'
#Here the matched template using template waveforms with mass ranges between 5 and 10 will be calculated 

#Here the waveform templates are generated for masses in range between 5 and 10, the procedure is similar to Problem 1
mass1 = numpy.arange(5 ,10 ,0.10 ) 
mass2 = mass1

for i in range(len(mass1)):
 
    m = mass1[i] 
    hp, hc = get_td_waveform(approximant="SEOBNRv4_opt",
                     mass1=m,
                     mass2=m,
                     delta_t=strain[ifo].delta_t, 
                     f_lower=15.0)
#Here the data is resized to match the teplate length    
hp.resize(len(strain[ifo])) 
template = hp.cyclic_time_shift(hp.start_time)
#Here the matched filter is performed by using the waveform as a template for the signal that should be filtered from noise
snr = matched_filter(template, strain[ifo], 
                     psd=psds[ifo], low_frequency_cutoff=15.0)
  
#Here the maximum SNR will be found, the procedure is similar to Problem 1
#Here SNR time series are croped by removing the first 4 and last 4 seconds of data 
snr_new = snr.crop(4 + 4, 4)
#Here the maximum absolute value of the SNR time series will be calculated and appended to the Snr_list and assigned to disctionary
Snr_new_list.append(max(abs(snr_new)))
snrs[ifo] = Snr__new_list

#Here the plot which shows SNR versus total mass of the system
#This will allow to see if the maximum SNR crosses 8

pylab.figure(figsize=[10, 4])
pylab.plot(mass1*2, snrs['SNR'], label= 'SNR')
pylab.legend('True')
pylab.ylabel('Signal-to-noise')
pylab.xlabel('mass')
pylab.show()





IndexError: ignored